In [ ]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

def load_dataset(name):
    if name == 'mnist':
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
        input_shape = (28, 28, 1)
    elif name == 'fashion_mnist':
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
        input_shape = (28, 28, 1)
    elif name == 'cifar10':
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        input_shape = (32, 32, 3)
    else:
        raise ValueError("Dataset not supported")

    # Normalize
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.

    # One-hot encode labels
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test, input_shape

def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def run_experiment(dataset_name, epochs):
    print(f"\n📦 Dataset: {dataset_name.upper()}, Epochs: {epochs}")
    x_train, y_train, x_test, y_test, input_shape = load_dataset(dataset_name)
    model = build_model(input_shape)

    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=64,
                        validation_data=(x_test, y_test), verbose=2)
    duration = time.time() - start_time

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")
    print(f"⏱️ Time Taken: {duration:.2f} seconds\n")
    return test_acc * 100, duration

In [ ]:
results = {}
for dataset in ['mnist', 'fashion_mnist', 'cifar10']:
    for epochs in [5, 10]:
        acc, duration = run_experiment(dataset, epochs)
        results[(dataset, epochs)] = (acc, duration)



📦 Dataset: MNIST, Epochs: 5
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 - 14s - 15ms/step - accuracy: 0.9575 - loss: 0.1359 - val_accuracy: 0.9892 - val_loss: 0.0349
Epoch 2/5
938/938 - 5s - 5ms/step - accuracy: 0.9879 - loss: 0.0376 - val_accuracy: 0.9841 - val_loss: 0.0498
Epoch 3/5
938/938 - 5s - 5ms/step - accuracy: 0.9916 - loss: 0.0260 - val_accuracy: 0.9919 - val_loss: 0.0224
Epoch 4/5
938/938 - 5s - 5ms/step - accuracy: 0.9937 - loss: 0.0200 - val_accuracy: 0.9909 - val_loss: 0.0280
Epoch 5/5
938/938 - 5s - 5ms/step - accuracy: 0.9949 - loss: 0.0156 - val_accuracy: 0.9924 - val_loss: 0.0258
✅ Test Accuracy: 99.24%
⏱️ Time Taken: 34.11 seconds


📦 Dataset: MNIST, Epochs: 10
Epoch 1/10
938/938 - 11s - 11ms/step - accuracy: 0.9583 - loss: 0.1300 - val_accuracy: 0.9834 - val_loss: 0.0471
Epoch 2/10
938/938 - 7s - 7ms/step - accuracy: 0.9874 - loss: 0.0394 - val_accuracy: 0.9910 - val_loss: 0.0267
Epoch 3/10
938/938 - 5s - 5ms/step - accuracy: 0.9919 - loss: 0.0257 - val_accuracy: 0.9904 - val_loss: 0.0298
Epoch 4/10
938/938 - 5s - 6ms

In [ ]:
print("\n📊 Summary Table:\n")
print(f"{'Dataset':<15}{'Epochs':<10}{'Accuracy (%)':<15}{'Time (s)':<10}")
print("-"*50)
for key, value in results.items():
    dataset, epochs = key
    acc, duration = value
    print(f"{dataset:<15}{epochs:<10}{acc:<15.2f}{duration:<10.2f}")



📊 Summary Table:

Dataset        Epochs    Accuracy (%)   Time (s)  
--------------------------------------------------
mnist          5         99.24          34.11     
mnist          10        99.43          63.62     
fashion_mnist  5         92.33          43.33     
fashion_mnist  10        92.48          75.97     
cifar10        5         73.70          33.43     
cifar10        10        73.90          60.64     
